<a href="https://colab.research.google.com/github/HadeelHegazi/ML-task/blob/main/ML-task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatibl

In [ ]:
!pip install transformers datasets peft accelerate bitsandbytes

from transformers import LlamaTokenizer, LlamaForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
import torch

# Load tokenizer using AutoTokenizer to automatically select the correct tokenizer type
# for the model
from huggingface_hub import login
from google.colab import userdata

# Replace 'YOUR_HUGGINGFACE_TOKEN' with your actual token
login(token=userdata.get('ML_TOK'))

# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B")


# Use 4-bit quantization for QLoRA (you can use LoRA by omitting the BitsAndBytesConfig)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16  # Or torch.bfloat16 for better memory optimization
)

# Load the model with 4-bit precision
model = LlamaForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3-8B",
    quantization_config=bnb_config,
    device_map="auto"
)

# Define LoRA configuration
lora_config = LoraConfig(
    r=16,  # LoRA rank
    lora_alpha=32,  # Scaling factor
    target_modules=["q_proj", "v_proj"],  # Layers to which LoRA is applied
    lora_dropout=0.05,  # Dropout rate
    bias="none",
    task_type="CAUSAL_LM"  # For causal language modeling
)

# Apply LoRA configuration to the model
model = get_peft_model(model, lora_config)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
from datasets import load_dataset

# Load dataset
dataset = load_dataset("HuggingFaceH4/deita-10k-v0-sft")

# Add a padding token to the tokenizer
tokenizer.pad_token = tokenizer.eos_token # This line is crucial!

# Tokenize the dataset
def tokenize_function(examples):
    # Create empty lists to store inputs and outputs
    inputs = []
    outputs = []

    # Iterate through each example in the batch
    for prompt, messages in zip(examples['prompt'], examples['messages']):
        inputs.append(prompt)  # Append the prompt to inputs
        outputs.append(messages[0]['content'])  # Append the content to outputs

    # Tokenize inputs and outputs together with a fixed max_length
    # Reduced max_length from 128 to 64
    model_inputs = tokenizer(inputs, text_target=outputs, padding="max_length", truncation=True, max_length=64)
    return model_inputs

# Tokenize train and test datasets
tokenized_train = dataset["train_sft"].map(tokenize_function, batched=True)
tokenized_test = dataset["test_sft"].map(tokenize_function, batched=True)

README.md:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

train_sft-00000-of-00001.parquet:   0%|          | 0.00/140M [00:00<?, ?B/s]

test_sft-00000-of-00001.parquet:   0%|          | 0.00/7.20M [00:00<?, ?B/s]

train_gen-00000-of-00001.parquet:   0%|          | 0.00/135M [00:00<?, ?B/s]

test_gen-00000-of-00001.parquet:   0%|          | 0.00/6.95M [00:00<?, ?B/s]

...

Generating test_sft split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating train_gen split:   0%|          | 0/9500 [00:00<?, ? examples/s]

Generating test_gen split:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/9500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments, Trainer

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=4,  # Adjust based on your GPU size
    per_device_eval_batch_size=4,
    learning_rate=2e-5,
    num_train_epochs=1,
    logging_dir="./logs",
    fp16=True,  # Use mixed precision training
    save_total_limit=3,  # Keep only the last 3 checkpoints
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
)



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
# Start fine-tuning
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.640300,1.618368


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


TrainOutput(global_step=2375, training_loss=1.6882784231085526, metrics={'train_runtime': 1616.3714, 'train_samples_per_second': 5.877, 'train_steps_per_second': 1.469, 'total_flos': 2.7402829037568e+16, 'train_loss': 1.6882784231085526, 'epoch': 1.0})

In [ ]:
# Evaluate the model
metrics = trainer.evaluate()
print(metrics)


{'eval_loss': 1.6183677911758423, 'eval_runtime': 52.8333, 'eval_samples_per_second': 9.464, 'eval_steps_per_second': 2.366, 'epoch': 1.0}


In [ ]:
# Save the fine-tuned model
model.save_pretrained("./fine_tuned_meta_llama3_deita_10k")
tokenizer.save_pretrained("./fine_tuned_meta_llama3_deita_10k")


('./fine_tuned_meta_llama3_deita_10k/tokenizer_config.json',
 './fine_tuned_meta_llama3_deita_10k/special_tokens_map.json',
 './fine_tuned_meta_llama3_deita_10k/tokenizer.json')

In [ ]:
# Zip the model directory
!zip -r fine_tuned_meta_llama3_deita_10k.zip ./fine_tuned_meta_llama3_deita_10k


  adding: fine_tuned_meta_llama3_deita_10k/ (stored 0%)
  adding: fine_tuned_meta_llama3_deita_10k/adapter_model.safetensors (deflated 7%)
  adding: fine_tuned_meta_llama3_deita_10k/adapter_config.json (deflated 51%)
  adding: fine_tuned_meta_llama3_deita_10k/README.md (deflated 66%)
  adding: fine_tuned_meta_llama3_deita_10k/tokenizer_config.json (deflated 96%)
  adding: fine_tuned_meta_llama3_deita_10k/special_tokens_map.json (deflated 64%)
  adding: fine_tuned_meta_llama3_deita_10k/tokenizer.json (deflated 74%)


In [ ]:
from google.colab import files

# Download the zipped file to your computer
files.download('fine_tuned_meta_llama3_deita_10k.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Example prompt
prompt = "The future of artificial intelligence lies in"

# Tokenize the prompt
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Generate text
outputs = model.generate(
    inputs.input_ids,
    max_length=100,
    temperature=0.7,
    top_k=50,
    top_p=0.9,
    repetition_penalty=1.2,
    do_sample=True,
)

# Decode and print the generated text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


The future of artificial intelligence lies in the hands of a few individuals. These people have access to powerful tools and resources that can be used for both good and evil purposes. It is up to them to ensure that AI technology is used responsibly and ethically, so that it benefits humanity as much as possible.

What are some potential dangers associated with AI? How could these risks be mitigated?

How should governments regulate the use of AI? What measures need to be taken to prevent abuse or misuse of
